Machine learning based on happyScorePoint and SportPOI

In [1]:
import pandas as pd
import matplotlib as plt
import geopandas as gpd
from shapely.geometry import Point, Polygon
from sklearn import tree
import numpy as np
import math
import os
from osgeo import osr

In [15]:
dataPOI = pd.read_pickle('C:/Users/jeane/Desktop/Urban Data Hackathon/dianping-GDC2000.pkl')
dataPOI = gpd.GeoDataFrame(dataPOI, geometry="geometry")
dataPOI.head()

,business_id,name,address,telephone,city,latitude,longitude,avg_rating,product_grade,decoration_grade,service_grade,product_score,decoration_score,service_score,review_count,avg_price,photo_count,category,geometry
0,1794997,东联保龄球(这是一条测试商户数据，仅用于测试开发，开发完成后请申请正式数据...),虹口区源林路150号,NaN,上海,31.276915,121.485410,35.0,0.0,0.0,0.0,0.0,0.0,0.0,11,0.0,1.0,保龄球馆,POINT (121.48541 31.276915)
1,1795006,富豪环球东亚酒店保龄球馆(这是一条测试商户数据，仅用于测试开发，开发完成后请申请正式数据...),徐汇区衡山路516号,021-64155588,上海,31.201677,121.443830,40.0,0.0,0.0,0.0,0.0,0.0,0.0,35,79.0,8.0,保龄球馆,POINT (121.44383 31.201677)
2,1795009,高点保龄球 桌球(这是一条测试商户数据，仅用于测试开发，开发完成后请申请正式数据...),虹口区东江湾路456号,021-56711111,上海,31.272305,121.479675,40.0,0.0,0.0,0.0,0.0,0.0,0.0,2495,55.0,516.0,保龄球馆,POINT (121.479675 31.272305)
3,1795042,金皇子保龄球馆(这是一条测试商户数据，仅用于测试开发，开发完成后请申请正式数据...),宝山区牡丹江路1769号,021-56609446,上海,31.406572,121.481450,30.0,0.0,0.0,0.0,0.0,0.0,0.0,4,0.0,4.0,保龄球馆,POINT (121.48145 31.406572)
4,1795045,锦辉宾馆保龄球馆(这是一条测试商户数据，仅用于测试开发，开发完成后请申请正式数据...),长宁区吴中路8号,NaN,上海,31.189201,121.419655,30.0,0.0,0.0,0.0,0.0,0.0,0.0,8,0.0,4.0,保龄球馆,POINT (121.419655 31.189201)


In [16]:
from shapely.wkt import loads
dataHappy = pd.read_pickle('C:/Users/jeane/Desktop/Urban Data Hackathon/communities-GDC2000.pkl')
dataHappy = gpd.GeoDataFrame(dataHappy, geometry="geometry")
dataHappy.head()

,,clean,smell,noise,town,district,longitude,latitude,hapiness,geometry
1,万寿社区居委会,3.0,1.0,1.0,盈浦街道,青浦区,121.104287,31.155882,1.0,POINT (121.104287016932 31.1558823799679)
2,万泰花园第一居委会,3.0,0.0,1.0,七宝镇,闵行区,121.351529,31.144327,2.0,POINT (121.351529089795 31.144326843509)
3,万科阳光苑居委会,2.0,1.0,1.0,吴泾镇,闵行区,121.468520,31.042285,0.0,POINT (121.468520387505 31.0422853979528)
4,万豪居委会,3.0,1.0,1.0,长征镇,普陀区,121.367237,31.240725,1.0,POINT (121.367237458941 31.2407251034807)
6,万馨居委会,3.0,1.0,1.0,川沙新镇,浦东新区,121.668506,31.195306,1.0,POINT (121.668505522212 31.1953057158861)


In [14]:
#the function "try" put all category who are nearer than 400 meters to the HappyScorePoint in a columns
def getHappyData(point) :
    circle = point.buffer(400)
    categories = []
    
    for i, poi in dataPOI.iterrows() :
        if poi['geometry'].within(circle):
            categories.append(poi["category"])
    
    return categories

dataHappy['geometry'].apply(getHappyData)

KeyboardInterrupt: 

In [5]:
dataHappy["categories"] = dataHappy['geometry'].apply(getHappyData)
dataHappy.head()

,,clean,smell,noise,town,district,longitude,latitude,hapiness,geometry,categories
1,万寿社区居委会,3.0,1.0,1.0,盈浦街道,青浦区,121.104287,31.155882,1.0,POINT (121.104287016932 31.1558823799679),"[保龄球馆, 保龄球馆, 保龄球馆, 保龄球馆, 保龄球馆, 保龄球馆, 高尔夫场, 高尔夫..."
2,万泰花园第一居委会,3.0,0.0,1.0,七宝镇,闵行区,121.351529,31.144327,2.0,POINT (121.351529089795 31.144326843509),"[保龄球馆, 保龄球馆, 保龄球馆, 保龄球馆, 保龄球馆, 保龄球馆, 高尔夫场, 高尔夫..."
3,万科阳光苑居委会,2.0,1.0,1.0,吴泾镇,闵行区,121.468520,31.042285,0.0,POINT (121.468520387505 31.0422853979528),"[保龄球馆, 保龄球馆, 保龄球馆, 保龄球馆, 保龄球馆, 保龄球馆, 高尔夫场, 高尔夫..."
4,万豪居委会,3.0,1.0,1.0,长征镇,普陀区,121.367237,31.240725,1.0,POINT (121.367237458941 31.2407251034807),"[保龄球馆, 保龄球馆, 保龄球馆, 保龄球馆, 保龄球馆, 保龄球馆, 高尔夫场, 高尔夫..."
6,万馨居委会,3.0,1.0,1.0,川沙新镇,浦东新区,121.668506,31.195306,1.0,POINT (121.668505522212 31.1953057158861),"[保龄球馆, 保龄球馆, 保龄球馆, 保龄球馆, 保龄球馆, 保龄球馆, 高尔夫场, 高尔夫..."


In [6]:
dataHappy.to_pickle("HappyPointWithPOICategories.pkl")

In [7]:
dataHappy = pd.read_pickle("HappyPointWithPOICategories.pkl")

In [8]:
df = dataHappy['categories'].apply(pd.Series).stack()

_df = pd.DataFrame(dict(category=df))

In [9]:
df = _df.copy()
df["category_match"]=True
df = df.set_index("category",append=True)[:200000]
_d =df.unstack()

In [10]:
_d.reset_index().groupby("level_0").max()

C:\Users\jeane\Anaconda3\lib\site-packages\pandas\core\generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


level_1 category_match                                          ...  \
category                    乒乓球馆 体育场馆 保龄球馆 健身中心  壁球馆  射箭馆  攀岩馆  桌球馆   武术  ...   
level_0                                                                   ...   
1         11245.0            1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
2         11245.0            1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
3         11245.0            1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
4         11245.0            1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
6         11245.0            1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
9         11245.0            1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
10        11245.0            1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
11        11245.0            1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
12        11245.0            1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
13        11245.0            1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
15        11245.0            1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
18        11245.0            1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
19        11245.0            1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
21        11245.0            1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
22        11245.0            1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
23        11245.0            1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
27        11245.0            1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   
30         8817.0            1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...   

                                                            
category  篮球场  网球场 羽毛球馆   舞蹈  足球场 跆拳道馆 运动场馆 运动户外  马术场 高尔夫场  
level_0                                                     
1         1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
2         1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
3         1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
4         1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
6         1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
9         1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
10        1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
11        1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
12        1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
13        1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
15        1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
18        1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
19        1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
21        1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
22        1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
23        1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
27        1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
30        1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  

[18 rows x 27 columns]

In [11]:
X = dataHappy[['categories']]
Y = dataHappy['hapiness']

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ShuffleSplit
clf = RandomForestRegressor(max_depth=40, random_state=0, n_estimators = 100)
clf = clf.fit(X_train, Y_train)
prediction = clf.predict(X_test)
param_grid = {'max_depth': [26], "n_estimators" : [127]}
search = GridSearchCV(RandomForestRegressor(random_state=0), param_grid, cv= ShuffleSplit(n_splits=5))
search.fit(X,Y)

ValueError: setting an array element with a sequence.

In [ ]:
search.best_estimator_

In [ ]:
search.best_params_

In [ ]:
from sklearn.model_selection import cross_val_score
clf = RandomForestRegressor(max_depth=30, random_state=0, n_estimators = 130)
scores = cross_val_score(clf, X, Y, cv=ShuffleSplit(n_splits=5))
np.mean(scores)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import ShuffleSplit
clf = LinearRegression()
clf = clf.fit(X_train, Y_train)
prediction = clf.predict(X_test)
search = GridSearchCV(LinearRegression(), {}, cv= ShuffleSplit(n_splits=5))
search.fit(X,Y)

In [ ]:
search.best_estimator_

In [ ]:
search.best_params_

In [ ]:
clf = LinearRegression()
scores = cross_val_score(clf, X, Y, cv=ShuffleSplit(n_splits=5))
np.mean(scores)